In [ ]:
# default_exp classification.modelling.classifiers

In [ ]:
# hide
import warnings

warnings.filterwarnings("ignore")

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# hide
from nbdev.showdoc import *
from nbdev.export import *
from nbdev.imports import Config as NbdevConfig

nbdev_path = str(NbdevConfig().path("nbs_path") / "data")
nbdev_path

'/Users/ayushman/Desktop/lightning_cv/nbs/data'

<IPython.core.display.Javascript object>

# Image Classification Classifiers
> Set of Classifiers for Image Classification.

In [ ]:
# export
from typing import *
import torch
from torch import nn
import torch.nn.functional as F

from timm.models.layers import create_classifier

from omegaconf import DictConfig
from fastcore.all import delegates, L

from lightning_cv.core.layers import *
from lightning_cv.core import Registry, ACTIVATION_REGISTERY

<IPython.core.display.Javascript object>

In [ ]:
# export
from fastcore.all import *

<IPython.core.display.Javascript object>

In [ ]:
# export
class GeneralClassifier(nn.Module):
    "A classification head w/ configurable global pooling and dropout that is similar to the one created by the timm library"

    @delegates(create_classifier)
    def __init__(
        self, num_features: int, num_classes: int, drop_rate: float = 0.0, **kwargs
    ):
        super(GeneralClassifier, self).__init__()
        self.global_pool, self.fc = create_classifier(
            num_features, num_classes, **kwargs
        )
        self.drop_rate = drop_rate

    def forward(self, x):
        x = self.global_pool(x)
        if self.drop_rate:
            x = F.dropout(x, p=float(self.drop_rate), training=self.training)
        x = self.fc(x)
        return x

    @classmethod
    def from_config(cls, cfg: DictConfig):
        return cls(**cfg)

<IPython.core.display.Javascript object>

In [ ]:
from lightning_cv.config import get_cfg
from lightning_cv.classification.modelling.backbones import create_cnn_backbone

cfg = get_cfg()
cfg.MODEL.BACKBONE.NAME = "TimmBackBoneBase"
cfg.MODEL.BACKBONE.ARGUMENTS = dict(
    model_name="efficientnet_b0", pretrained=True, cut=None, act_layer=None
)

<IPython.core.display.Javascript object>

In [ ]:
backbone = create_cnn_backbone(cfg)
features = num_features_model(backbone)

cfg.MODEL.CLASSIFIER.ARGUMENTS = dict(
    num_features=features, num_classes=1000, drop_rate=0.25, pool_type="avg"
)
tst = GeneralClassifier.from_config(cfg.MODEL.CLASSIFIER.ARGUMENTS)

print(tst)

mods = list(tst.children())
test_eq(len(mods), 2)

GeneralClassifier(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=True)
  (fc): Linear(in_features=1280, out_features=1000, bias=True)
)


<IPython.core.display.Javascript object>

In [ ]:
backbone = create_cnn_backbone(cfg)
features = num_features_model(backbone)

cfg.MODEL.CLASSIFIER.ARGUMENTS = dict(
    num_features=features, num_classes=1000, drop_rate=0.25, pool_type="catavgmax"
)
tst = GeneralClassifier.from_config(cfg.MODEL.CLASSIFIER.ARGUMENTS)

print(tst)

mods = list(tst.children())
test_eq(len(mods), 2)

GeneralClassifier(
  (global_pool): SelectAdaptivePool2d (pool_type=catavgmax, flatten=True)
  (fc): Linear(in_features=2560, out_features=1000, bias=True)
)


<IPython.core.display.Javascript object>

In [ ]:
# export
class FastaiClassifier(nn.Sequential):
    "A classification head same as head generated from : https://github.com/fastai/fastai/blob/master/fastai/vision/learner.py#L76"

    def __init__(
        self,
        num_features: int,
        num_classes: int,
        lin_ftrs: Optional[List] = None,
        ps: float = 0.5,
        concat_pool: bool = True,
        first_bn: bool = True,
        bn_final: bool = False,
        lin_first: bool = False,
        act_layer: str = "ReLU",
    ):

        # if using concat_pool then mult the number of features
        if concat_pool:
            num_features *= 2

        # features of the linear layers
        lin_ftrs = (
            [num_features, 512, num_classes]
            if lin_ftrs is None
            else [num_features] + lin_ftrs + [num_classes]
        )

        bns = [first_bn] + [True] * len(lin_ftrs[1:])

        ps = L(ps)

        if len(ps) == 1:
            ps = [ps[0] / 2] * (len(lin_ftrs) - 2) + ps

        act_layer = [ACTIVATION_REGISTERY.get(act_layer)(inplace=True)]
        actns = act_layer * (len(lin_ftrs) - 2) + [None]
        pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)

        layers = [pool, nn.Flatten()]

        if lin_first:
            layers.append(nn.Dropout(ps.pop(0)))

        for ni, no, bn, p, actn in zip(lin_ftrs[:-1], lin_ftrs[1:], bns, ps, actns):
            layers += LinBnDrop(ni, no, bn=bn, p=p, act=actn, lin_first=lin_first)

        if lin_first:
            layers.append(nn.Linear(lin_ftrs[-2], num_classes))

        if bn_final:
            layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))

        super(FastaiClassifier, self).__init__(*layers)

    @classmethod
    def from_config(cls, cfg: DictConfig):
        return cls(**cfg)

<IPython.core.display.Javascript object>

The head begins with with a `AdaptiveConcatPool2d` (adapted from fastai) if `concat_pool=True` otherwise, it uses traditional average pooling. Then it uses a Flatten layer before going on blocks of BatchNorm, Dropout and Linear layers `(if lin_first=True, those are Linear, BatchNorm, Dropout)`.

Those blocks start at `num_features`, then every element of `lin_ftrs` (defaults to [512]) and end at `num_classes`. `ps` is a list of probabilities used for the dropouts (if you only pass 1, it will use half the value then that value as many times as necessary).

If `first_bn=True`, a BatchNorm added just after the pooling operations. If `bn_final=True`, a final BatchNorm layer is added.

Activation functions are added after every blocks of BatchNorm, Dropout and Linear layers. The function is inferred from `ACTIVATION_REGISTERY` using `act_layer` (defaults to ReLU).

In [ ]:
tst = FastaiClassifier(features, 10)
tst

FastaiClassifier(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): BatchNorm1d(2560, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=2560, out_features=512, bias=False)
  (5): ReLU(inplace=True)
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=512, out_features=10, bias=False)
)

<IPython.core.display.Javascript object>

In [ ]:
# hide
mods = list(tst.children())
test_eq(len(mods), 9)
assert isinstance(mods[2], nn.BatchNorm1d)
assert isinstance(mods[-1], nn.Linear)

tst = FastaiClassifier(5, 10, lin_first=True)
mods = list(tst.children())
test_eq(len(mods), 8)
assert isinstance(mods[2], nn.Dropout)

tst = FastaiClassifier(5, 10, first_bn=False)
mods = list(tst.children())
test_eq(len(mods), 8)
assert isinstance(mods[2], nn.Dropout)

tst = FastaiClassifier(5, 10, concat_pool=True)
modes = list(tst.children())
test_eq(modes[4].in_features, 10)

tst = FastaiClassifier(5, 10, concat_pool=False)
modes = list(tst.children())
test_eq(modes[4].in_features, 5)

<IPython.core.display.Javascript object>

In [ ]:
tst = FastaiClassifier(features, 10, act_layer="Mish", lin_ftrs=[512, 155])
tst

FastaiClassifier(
  (0): AdaptiveConcatPool2d(
    (ap): AdaptiveAvgPool2d(output_size=1)
    (mp): AdaptiveMaxPool2d(output_size=1)
  )
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): BatchNorm1d(2560, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.25, inplace=False)
  (4): Linear(in_features=2560, out_features=512, bias=False)
  (5): Mish()
  (6): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.25, inplace=False)
  (8): Linear(in_features=512, out_features=155, bias=False)
  (9): Mish()
  (10): BatchNorm1d(155, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Dropout(p=0.5, inplace=False)
  (12): Linear(in_features=155, out_features=10, bias=False)
)

<IPython.core.display.Javascript object>

## Classifier Registery 

In [ ]:
# export
CLASSIFICATION_CLASSIFIER_REGISTRY = Registry("Classification Classifiers")

CLASSIFICATION_CLASSIFIER_REGISTRY.__doc__ = (
    "Registry of Classifiers for Image Classification"
)
CLASSIFICATION_CLASSIFIER_REGISTRY.register(FastaiClassifier)
CLASSIFICATION_CLASSIFIER_REGISTRY.register(GeneralClassifier)

<IPython.core.display.Javascript object>

In [ ]:
show_doc(CLASSIFICATION_CLASSIFIER_REGISTRY)

<h4 id="Classification Classifiers" class="doc_header"><code>Classification Classifiers</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Registry of Classifiers for Image Classification

<IPython.core.display.Javascript object>

In [ ]:
# collapse-output
CLASSIFICATION_CLASSIFIER_REGISTRY

Registry of Classification Classifiers:
╒═══════════════════╤══════════════════════════════════════╕
│ Names             │ Objects                              │
╞═══════════════════╪══════════════════════════════════════╡
│ FastaiClassifier  │ <class '__main__.FastaiClassifier'>  │
├───────────────────┼──────────────────────────────────────┤
│ GeneralClassifier │ <class '__main__.GeneralClassifier'> │
╘═══════════════════╧══════════════════════════════════════╛

<IPython.core.display.Javascript object>

In [ ]:
# export
def create_classifier_head(cfg: DictConfig):
    "instante an obj from `CNN_CLASSIFIER_REGISTRY` registery using lightning_cv.config"
    classifier_cls = CLASSIFICATION_CLASSIFIER_REGISTRY.get(cfg.MODEL.CLASSIFIER.NAME)
    classifier = classifier_cls.from_config(cfg.MODEL.CLASSIFIER.ARGUMENTS)
    return classifier

<IPython.core.display.Javascript object>

Create a `classification_head` from `lightning_cv.config` -

In [ ]:
from lightning_cv.config import get_cfg
from omegaconf import OmegaConf

cfg = get_cfg()
cfg.MODEL.CLASSIFIER.NAME = "GeneralClassifier"
print(OmegaConf.to_yaml(cfg.MODEL))

BACKBONE:
  NAME: TimmBackBoneBase
  ARGUMENTS:
    model_name: efficientnet_b0
    cut: null
    act_layer: null
    pretrained: true
    drop_path_rate: 0.3
CLASSIFIER:
  NAME: GeneralClassifier
  ARGUMENTS:
    num_features: ???
    num_classes: ???
    drop_rate: 0.25
    pool_type: avg
    use_conv: false



<IPython.core.display.Javascript object>

Building your model from config -

In [ ]:
from lightning_cv.classification.modelling.backbones import create_cnn_backbone

backbone = create_cnn_backbone(cfg)
features = num_features_model(backbone)

cfg.MODEL.CLASSIFIER.NAME = "GeneralClassifier"
cfg.MODEL.CLASSIFIER.ARGUMENTS.num_features = features
cfg.MODEL.CLASSIFIER.ARGUMENTS.num_classes = 10

classifier = create_classifier_head(cfg)
print(classifier)

GeneralClassifier(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=True)
  (fc): Linear(in_features=1280, out_features=10, bias=True)
)


<IPython.core.display.Javascript object>

In [ ]:
inputs = torch.randn(1, 3, 255, 255)
tst = nn.Sequential(backbone, classifier)
output = tst(inputs)
test_eq(output.shape, [1, 10])

<IPython.core.display.Javascript object>

You can also create your custom classifier head like so -

In [ ]:
@CLASSIFICATION_CLASSIFIER_REGISTRY.register()
class MyCustomClassifier(nn.Sequential):
    def __init__(self, num_features: int, num_classes: int):
        fc1 = nn.Linear(num_features, num_classes, bias=False)
        super(MyCustomClassifier, self).__init__(fc1)

    @classmethod
    def from_config(cls, cfg: DictConfig):
        return cls(**cfg)

<IPython.core.display.Javascript object>

> Important: You custom classifier must have a `.from_config` classmethod with takes in a Omegaconf config and instantiates the class.

In [ ]:
cfg.MODEL.CLASSIFIER.NAME = "MyCustomClassifier"
cfg.MODEL.CLASSIFIER.ARGUMENTS = dict(num_features=5, num_classes=10)
create_classifier_head(cfg)

MyCustomClassifier(
  (0): Linear(in_features=5, out_features=10, bias=False)
)

<IPython.core.display.Javascript object>

In [ ]:
# hide
notebook2script()

Converted 00_config.ipynb.
Converted 00a_core.utils.common.ipynb.
Converted 00b_core.utils.data.ipynb.
Converted 00c_core.optim.ipynb.
Converted 00d_core.schedules.ipynb.
Converted 00e_core.layers.ipynb.
Converted 01a_classification.data.transforms.ipynb.
Converted 01b_classification.data.datasets.ipynb.
Converted 01c_classification.modelling.backbones.ipynb.
Converted 01d_classification.modelling.classifiers.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>